# Imports

In [ ]:
import itertools as it
import os
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Focus

In [ ]:
filename = "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/230726_ultrarainbow_40x40_zstack_nocy7_fov1.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.sizes

In [ ]:
nd2.metadata["channels"]

In [ ]:
display_image(
    nd2.get_frame_2D(v=0, t=0, z=8, c=1)[2000:2500, 1000:1500], scale=1, downsample=1
)

In [ ]:
focus_func = lap4
metric = [
    lap4(nd2.get_frame_2D(v=0, t=0, z=z, c=1)[2000:2500, 1000:1500])
    for z in trange(nd2.sizes["z"])
]

In [ ]:
hv.Curve(metric)

In [ ]:
focus_funcs = [lap4, gra6, gra7, sta3]
d = [
    np.array(
        [
            focus_func(nd2.get_frame_2D(v=0, t=0, z=z, c=0)[2000:2500, 1000:1500])
            for z in trange(nd2.sizes["z"])
        ]
    )
    for focus_func in tqdm(focus_funcs)
]

In [ ]:
hv.Overlay(
    [
        hv.Curve(dd / dd.max(), label=focus_func.__name__)
        for dd, focus_func in zip(d, focus_funcs)
    ]
)

In [ ]:
hv.Overlay(
    [
        hv.Curve(dd / dd.max(), label=focus_func.__name__)
        for dd, focus_func in zip(d, focus_funcs)
    ]
)

In [ ]:
d2 = [
    np.array(
        [
            gra7(nd2.get_frame_2D(v=0, t=0, z=z, c=c)[2000:2500, 1000:1500])
            for z in trange(nd2.sizes["z"])
        ]
    )
    for c in trange(nd2.sizes["c"])
]

In [ ]:
hv.Overlay(
    [
        hv.Curve(dd / dd.max(), label=channel)
        for dd, channel in zip(d2, nd2.metadata["channels"])
    ]
)

In [ ]:
# histogram equilibrization (better background subtraction/normalization)
# use bead moments to calculate focus, allow moving-window averaging
# find focus offset in PFS units
# look for spatial variation of focus

# Bead segmentation

In [ ]:
from sklearn.neighbors import KDTree


def segment_puncta(img, low_sigma=1, high_sigma=50, expand=2):
    img_dog = skimage.filters.difference_of_gaussians(img, low_sigma, high_sigma)
    img_mask = img_dog > skimage.filters.threshold_otsu(img_dog)
    img_labels = skimage.measure.label(img_mask)
    if expand is not None:
        img_labels = skimage.segmentation.expand_labels(img_labels, expand)
    return img_labels


def measure_puncta(img, img_labels=None, min_dist=10, max_intensity=3):
    if img_labels is None:
        img_labels = segment_points(img)
    background = np.median(img[img_labels == 0])
    # subtract background (note that some pixels may be negative)
    img_bgsub = img - background
    # normalize image
    img_bgsub /= img_bgsub.max()
    df = pd.DataFrame(
        skimage.measure.regionprops_table(
            img_labels,
            img_bgsub,
            properties=(
                "label",
                "centroid_weighted",
                "area",
                "moments_weighted_central",
            ),
        )
    )
    df["radius"] = np.sqrt(
        df["centroid_weighted-0"] ** 2 + df["centroid_weighted-1"] ** 2
    )
    X = np.stack((df["centroid_weighted-0"], df["centroid_weighted-1"]), axis=1)
    kdtree = KDTree(X)
    dists, idxs = kdtree.query(X, k=2)
    # get nearest non-identical neighbor
    dists = dists[:, 1]
    idxs = idxs[:, 1]
    overcrowding_mask = np.full(len(df), True)
    # filter out points with nearest neighbors closer than min_dist
    overcrowding_mask[idxs[dists < min_dist]] = False
    df_filtered = df[
        overcrowding_mask & (df["moments_weighted_central-0-0"] < max_intensity)
    ]
    return df_filtered

In [ ]:
filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cy5.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.sizes

In [ ]:
%%time
img = nd2.get_frame_2D(v=0, t=0, z=0, c=0)
df = measure_points(img)

In [ ]:
plt.hist(df["area"], bins=100);

In [ ]:
plt.hist(dists, bins=100);

In [ ]:
df.hvplot.hist("moments_weighted_central-0-0", bins=100)

In [ ]:
df

In [ ]:
plt.figure(figsize=(20, 20), dpi=200)
plt.imshow(img_bgsub, vmin=0, vmax=0.1)
plt.plot(
    df["centroid_weighted-1"],
    df["centroid_weighted-0"],
    marker="o",
    mfc="none",
    c="red",
    markersize=4,
    lw=0,
    markeredgewidth=0.5,
);

In [ ]:
plt.figure(figsize=(20, 20), dpi=200)
plt.imshow(skimage.color.label2rgb(img_labels, img * 50))
plt.plot(
    df2["centroid_weighted-1"],
    df2["centroid_weighted-0"],
    marker="o",
    mfc="none",
    c="red",
    markersize=4,
    lw=0,
    markeredgewidth=0.5,
);

In [ ]:
df3 = df2[df2["moments_weighted_central-0-0"] > 3]
plt.figure(figsize=(20, 20), dpi=300)
plt.imshow(skimage.color.label2rgb(img_labels, img * 50))
plt.plot(
    df3["centroid_weighted-1"],
    df3["centroid_weighted-0"],
    marker="o",
    mfc="none",
    c="red",
    markersize=4,
    lw=0,
    markeredgewidth=0.5,
);

# Focus visualizations

## Spatial

In [ ]:
filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_64fov_cy5.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
img = nd2.get_frame_2D(v=0, t=0, z=0, c=0)

In [ ]:
df = measure_puncta(img)

In [ ]:
df2 = df.assign(
    width=df["moments_weighted_central-2-2"] / df["moments_weighted_central-0-0"]
)
df2 = df2[df2["width"].between(0, 20)]
df2.hvplot.scatter("centroid_weighted-1", "centroid_weighted-0", color="width")

## Z stack

In [ ]:
filename = "/home/jqs1/scratch/jqs1/microscopy/230728/calibration/230728_ultrarainbow_51zstack_1fov_nocy7.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.sizes

In [ ]:
focus_metrics = [
    np.array(
        [
            focus.gra7(nd2.get_frame_2D(v=0, t=0, z=z, c=c))
            for z in trange(nd2.sizes["z"])
        ]
    )
    for c in trange(nd2.sizes["c"])
]

In [ ]:
hv.Overlay(
    [
        hv.Curve(channel_focus / channel_focus.max(), label=channel)
        for channel_focus, channel in zip(focus_metrics, nd2.metadata["channels"])
    ]
)

In [ ]:
hv.HoloMap(
    {
        z: ui.RevImage(nd2.get_frame_2D(v=0, t=0, z=z, c=0)[:500, :500])
        for z in trange(20)
    }
)

In [ ]:
display_image(nd2.get_frame_2D(v=0, t=0, z=5, c=0), scale=0.9999)

In [ ]:
img_labels = segment_puncta(nd2.get_frame_2D(v=0, t=0, z=20, c=1))
dfs = {
    z: measure_puncta(nd2.get_frame_2D(v=0, t=0, z=z, c=1), img_labels)
    for z in trange(nd2.sizes["z"])
}

In [ ]:
hv.HoloMap(
    {
        z: dfs[z].hvplot.scatter(
            "centroid_weighted-1",
            "centroid_weighted-0",
            color="moments_weighted_central-0-0",
        )
        for z in range(nd2.sizes["z"])
    }
)

In [ ]:
hv.Overlay(
    [
        hv.Curve(channel_focus / channel_focus.max(), label=channel)
        for (z, channel in zip(dfs.items(), ())
    ]
)

# Translation

In [ ]:
# Cy7 translation, GFP/YFP translations
# better SNR Cy5?
# measure focus offset in PFS units, verify

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/230726_ultrarainbow_40x40_Cy5-EM.nd2"
filename = "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/230726_ultrarainbow_40x40_zstack_nocy7_fov1.nd2"
nd2 = nd2reader.ND2Reader(filename)

In [ ]:
nd2.metadata["channels"]

In [ ]:
img = nd2.get_frame_2D(v=0, t=0, z=8, c=1)

In [ ]:
import imageio.v3 as iio

In [ ]:
iio.imwrite(
    "/home/jqs1/scratch/jqs1/microscopy/230726/calibration/beads_Cy5.tiff",
    img,
    plugin="pillow",
    extension=".tiff",
)